In [33]:
import pandas as pd
import plotly.graph_objects as go
import requests
from requests.auth import HTTPBasicAuth
import gridstatusio
import warnings


from gridstatusio import GridStatusClient
gridstatusio.__version__
import time
from pathlib import Path

In [28]:
grid_status_api_key = "0438e7b9f460415a865fe6893e334442"
QUERY_LIMIT = 50
grid_status_dataset = "caiso_lmp_day_ahead_hourly"


login_url = 'https://api.watttime.org/login'
rsp = requests.get(login_url, auth=HTTPBasicAuth('AAAK', 'Pelikan99@'))
TOKEN = rsp.json()['token']
wattime_region = "CAISO_NORTH"
hour_list = [hour for hour in range(24) for _ in range(12)]


In [29]:
dates   =  {'Feb':["2024-02-09","2024-02-10","2024-02-11","2024-02-12","2024-02-13","2024-02-14","2024-02-15","2024-02-16","2024-02-17","2024-02-18"],
            'May':["2024-05-09","2024-05-10","2024-05-11","2024-05-12","2024-05-13","2024-05-14","2024-05-15","2024-05-16","2024-05-17","2024-05-18"], 
            'Aug':["2024-08-09","2024-08-10","2024-08-11","2024-08-12","2024-08-13","2024-08-14","2024-08-15","2024-08-16","2024-08-17","2024-08-18"],
            'Nov':["2024-11-09","2024-11-10","2024-11-11","2024-11-12","2024-11-13","2024-11-14","2024-11-15","2024-11-16","2024-11-17","2024-11-18"],
            }

In [30]:
client = GridStatusClient(api_key=grid_status_api_key)

In [32]:
warnings.filterwarnings("ignore")
for key, value in dates.items():
    for i in range(1,len(value)-1):
        time.sleep(3)  # wait 1 second between requests
        print(value[i-1],value[i],value[i+1])
        price_df = client.get_dataset(
            dataset= grid_status_dataset,
            start=value[i],  # start from beginning of data
            end=value[i+1],  # end at most recent data,
            filter_column="location",
            filter_value="RAINBOW_1_N001",
            # filter_value="NSP.S_RIDGE1",
            limit=QUERY_LIMIT,
        )

        url = "https://api.watttime.org/v3/historical"


        headers = {"Authorization": f"Bearer {TOKEN}"}
        params = {
            "region": wattime_region,
            "start": f"{value[i-1]}T00:00:00-0800",
            "end": f"{value[i+1]}T00:00:00-0800",
            "signal_type": "co2_moer",
        }
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        r = response.json()

        df = pd.DataFrame(r['data'])
        price_emissions_df = pd.DataFrame()
        emission_data = response.json()['data']
        emission_df = pd.DataFrame(emission_data)
        end_id = emission_df.loc[emission_df['point_time']==f"{value[i]}T00:00:00+00:00"].index.to_list()[0]
        start_id = emission_df.loc[emission_df['point_time']==f"{value[i+1]}T00:00:00+00:00"].index.to_list()[0]
        emission_df = emission_df[end_id:start_id]
        emission_data = emission_df['value']
        emission_df[start_id:end_id]


        price_emissions_df['hour'] = hour_list
        repeated_prices = price_df['lmp'].repeat(12).reset_index(drop=True)

        price_emissions_df['price'] = price_df['lmp'].repeat(12).reset_index(drop=True)
        price_emissions_df['emissions'] = emission_df['value'].values

        price_emissions_df.to_csv(f"{value[i]}.csv")


2024-02-09 2024-02-10 2024-02-11
Fetching Page 1...Done in 0.12 seconds. 
Total rows: 24/50 (48.0% of limit)

Total number of rows: 24
2024-02-10 2024-02-11 2024-02-12
Fetching Page 1...Done in 0.3 seconds. 
Total rows: 24/50 (48.0% of limit)

Total number of rows: 24
2024-02-11 2024-02-12 2024-02-13
Fetching Page 1...Done in 0.14 seconds. 
Total rows: 24/50 (48.0% of limit)

Total number of rows: 24
2024-02-12 2024-02-13 2024-02-14
Fetching Page 1...Done in 0.46 seconds. 
Total rows: 24/50 (48.0% of limit)

Total number of rows: 24
2024-02-13 2024-02-14 2024-02-15
Fetching Page 1...Done in 0.15 seconds. 
Total rows: 24/50 (48.0% of limit)

Total number of rows: 24
2024-02-14 2024-02-15 2024-02-16
Fetching Page 1...Done in 0.18 seconds. 
Total rows: 24/50 (48.0% of limit)

Total number of rows: 24
2024-02-15 2024-02-16 2024-02-17
Fetching Page 1...Done in 0.2 seconds. 
Total rows: 24/50 (48.0% of limit)

Total number of rows: 24
2024-02-16 2024-02-17 2024-02-18
Fetching Page 1...Done i

In [34]:

cwd = Path.cwd()

for csv_path in cwd.glob("2024*.csv"):
    # 1) Read the messy file -------------------------------------------
    df = pd.read_csv(
        csv_path,
        header=None,                 # there is no formal header row
        skiprows=1,                  # skip the text line ",hour,price,emissions"
        names=["idx", "hour", "price", "emissions"]
    )

    # 2) Transform ------------------------------------------------------
    df["time"] = df["idx"] * 5
    df = df[["time", "price", "emissions"]]   # drop the hour column

    # 3) Over-write the original file ----------------------------------
    df.to_csv(csv_path, index=False)
    print(f"✔  updated {csv_path.name}")


✔  updated 2024-11-16.csv
✔  updated 2024-11-17.csv
✔  updated 2024-11-15.csv
✔  updated 2024-11-14.csv
✔  updated 2024-11-10.csv
✔  updated 2024-11-11.csv
✔  updated 2024-11-13.csv
✔  updated 2024-11-12.csv
✔  updated 2024-05-14.csv
✔  updated 2024-05-15.csv
✔  updated 2024-05-17.csv
✔  updated 2024-05-16.csv
✔  updated 2024-05-12.csv
✔  updated 2024-05-13.csv
✔  updated 2024-05-11.csv
✔  updated 2024-05-10.csv
✔  updated 2024-02-14.csv
✔  updated 2024-02-15.csv
✔  updated 2024-02-17.csv
✔  updated 2024-02-16.csv
✔  updated 2024-02-12.csv
✔  updated 2024-02-13.csv
✔  updated 2024-02-11.csv
✔  updated 2024-02-10.csv
✔  updated 2024-08-16.csv
✔  updated 2024-08-17.csv
✔  updated 2024-08-15.csv
✔  updated 2024-08-14.csv
✔  updated 2024-08-10.csv
✔  updated 2024-08-11.csv
✔  updated 2024-08-13.csv
✔  updated 2024-08-12.csv
